In [65]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [33]:
transactions = pd.read_csv('transaction_data.csv')
purchase= pd.read_csv('purchase_behaviour.csv')


In [4]:
transactions.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [5]:
transactions['PROD_NBR'].value_counts()

PROD_NBR
102    3304
108    3296
33     3269
112    3268
75     3265
       ... 
11     1431
76     1430
98     1419
29     1418
72     1410
Name: count, Length: 114, dtype: int64

In [51]:
sales= transactions.groupby('PROD_NBR')['TOT_SALES'].sum()
sales

PROD_NBR
1       8125.8
2      22944.4
3      28308.4
4      40352.0
5       8331.0
        ...   
110     5367.5
111     9135.0
112    26149.2
113    27853.0
114    27567.8
Name: TOT_SALES, Length: 114, dtype: float64

In [52]:
sales=sales.sort_values(ascending=False).head(3)
sales

PROD_NBR
4     40352.0
14    36367.6
16    34804.2
Name: TOT_SALES, dtype: float64

In [26]:
prods= sales.index.tolist()
sold= sales.values.tolist()

In [29]:
top_prods= transactions[transactions['PROD_NBR'].isin(prods)]['PROD_NAME'].unique()

In [30]:
top_prods

array(['Smiths Crinkle Chips Salt & Vinegar 330g',
       'Dorito Corn Chp     Supreme 380g',
       'Smiths Crnkle Chip  Orgnl Big Bag 380g'], dtype=object)

In [32]:
print("The top 3 products and their total sales are: ")
for i,j in zip(top_prods,sold):
    print(i," : ",j)    

The top 3 products and their total sales are: 
Smiths Crinkle Chips Salt & Vinegar 330g  :  40352.0
Dorito Corn Chp     Supreme 380g  :  36367.6
Smiths Crnkle Chip  Orgnl Big Bag 380g  :  34804.200000000004


In [34]:
merged= pd.merge(transactions,purchase, on='LYLTY_CARD_NBR', how='left')

In [36]:
merged.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,LIFESTAGE,PREMIUM_CUSTOMER
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0,YOUNG SINGLES/COUPLES,Premium
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3,MIDAGE SINGLES/COUPLES,Budget
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9,MIDAGE SINGLES/COUPLES,Budget
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0,MIDAGE SINGLES/COUPLES,Budget
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8,MIDAGE SINGLES/COUPLES,Budget


In [85]:
merged['LIFESTAGE'].value_counts()

LIFESTAGE
OLDER SINGLES/COUPLES     54479
RETIREES                  49763
OLDER FAMILIES            48596
YOUNG FAMILIES            43592
YOUNG SINGLES/COUPLES     36377
MIDAGE SINGLES/COUPLES    25110
NEW FAMILIES               6919
Name: count, dtype: int64

In [69]:
grouped = merged.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'])['TOT_SALES'].sum()
grouped= grouped.reset_index()
print(grouped)

                 LIFESTAGE PREMIUM_CUSTOMER  TOT_SALES
0   MIDAGE SINGLES/COUPLES           Budget   35514.80
1   MIDAGE SINGLES/COUPLES       Mainstream   90803.85
2   MIDAGE SINGLES/COUPLES          Premium   58432.65
3             NEW FAMILIES           Budget   21928.45
4             NEW FAMILIES       Mainstream   17013.90
5             NEW FAMILIES          Premium   11491.10
6           OLDER FAMILIES           Budget  168363.25
7           OLDER FAMILIES       Mainstream  103445.55
8           OLDER FAMILIES          Premium   81958.40
9    OLDER SINGLES/COUPLES           Budget  136769.80
10   OLDER SINGLES/COUPLES       Mainstream  133393.80
11   OLDER SINGLES/COUPLES          Premium  132263.15
12                RETIREES           Budget  113147.80
13                RETIREES       Mainstream  155677.05
14                RETIREES          Premium   97646.05
15          YOUNG FAMILIES           Budget  139345.85
16          YOUNG FAMILIES       Mainstream   92788.75
17        

In [70]:
sorted_grouped= grouped.sort_values(by= 'TOT_SALES',ascending=False)


In [72]:
model_lifestage = ols('TOT_SALES ~ C(LIFESTAGE)', data=sorted_grouped).fit()
anova_table_lifestage = sm.stats.anova_lm(model_lifestage, typ=2)
model_premium = ols('TOT_SALES ~ C(PREMIUM_CUSTOMER)', data=sorted_grouped).fit()
anova_table_premium = sm.stats.anova_lm(model_premium, typ=2)

In [73]:
print(anova_table_lifestage)

                    sum_sq    df         F    PR(>F)
C(LIFESTAGE)  3.042608e+10   6.0  4.190129  0.012769
Residual      1.694320e+10  14.0       NaN       NaN


In [75]:
print(anova_table_premium)

                           sum_sq    df         F    PR(>F)
C(PREMIUM_CUSTOMER)  4.439069e+09   2.0  0.930618  0.412473
Residual             4.293020e+10  18.0       NaN       NaN


In [78]:
grouped

,LIFESTAGE,PREMIUM_CUSTOMER,TOT_SALES
0,MIDAGE SINGLES/COUPLES,Budget,35514.80
1,MIDAGE SINGLES/COUPLES,Mainstream,90803.85
2,MIDAGE SINGLES/COUPLES,Premium,58432.65
3,NEW FAMILIES,Budget,21928.45
4,NEW FAMILIES,Mainstream,17013.90
5,NEW FAMILIES,Premium,11491.10
6,OLDER FAMILIES,Budget,168363.25
7,OLDER FAMILIES,Mainstream,103445.55
8,OLDER FAMILIES,Premium,81958.40
9,OLDER SINGLES/COUPLES,Budget,136769.80


In [79]:
sorted_grouped

,LIFESTAGE,PREMIUM_CUSTOMER,TOT_SALES
6,OLDER FAMILIES,Budget,168363.25
19,YOUNG SINGLES/COUPLES,Mainstream,157621.60
13,RETIREES,Mainstream,155677.05
15,YOUNG FAMILIES,Budget,139345.85
9,OLDER SINGLES/COUPLES,Budget,136769.80
10,OLDER SINGLES/COUPLES,Mainstream,133393.80
11,OLDER SINGLES/COUPLES,Premium,132263.15
12,RETIREES,Budget,113147.80
7,OLDER FAMILIES,Mainstream,103445.55
14,RETIREES,Premium,97646.05


In [81]:
grouped[grouped['LIFESTAGE']=='OLDER FAMILIES']['TOT_SALES'].mean()

np.float64(117922.39999999998)

In [82]:
grouped[grouped['LIFESTAGE']=='OLDER SINGLES/COUPLES']['TOT_SALES'].mean()

np.float64(134142.25)

In [83]:
grouped[grouped['LIFESTAGE']=='YOUNG SINGLES/COUPLES']['TOT_SALES'].mean()

np.float64(86801.76666666668)

In [88]:

grouped[grouped['LIFESTAGE']=='YOUNG FAMILIES']['TOT_SALES'].mean()

np.float64(105386.7)

In [89]:
grouped[grouped['LIFESTAGE']=='MIDAGE SINGLES/COUPLES']['TOT_SALES'].mean()

np.float64(61583.76666666667)

In [90]:
grouped[grouped['LIFESTAGE']=='NEW FAMILIES']['TOT_SALES'].mean()

np.float64(16811.15)